In [1]:
import pandas as pd

In [2]:
goalscorers = pd.read_csv('goalscorers.csv')
results = pd.read_csv('results.csv')
shootouts = pd.read_csv('shootouts.csv')

In [3]:
# Function to determine the winning team
def determine_winner(row):
    if row['home_score'] == row['away_score']:
        return 'TIE'
    elif row['home_score'] > row['away_score']:
        return row['home_team']
    else:
        return row['away_team']

# Add a new column 'winner' using the determine_winner function
results['game_winner'] = results.apply(determine_winner, axis=1)
results.head(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,game_winner
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,TIE
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland


In [4]:
# Rename column to specify shootout winner in case game was a tie
shootouts = shootouts.rename(columns={'winner': 'penalty_winner'})

# Merge results and shootouts
merged_results = results.merge(shootouts, how='left', on=['date', 'home_team', 'away_team'])
merged_results.head(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,game_winner,penalty_winner,first_shooter
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,TIE,NaN,NaN
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,NaN,NaN
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland,NaN,NaN


In [5]:
# Add the "Shootout" column
merged_results['Shootout'] = merged_results['penalty_winner'].notnull()
merged_results.head(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,game_winner,penalty_winner,first_shooter,Shootout
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,TIE,NaN,NaN,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,NaN,NaN,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland,NaN,NaN,False


In [6]:
# Add extra columns based on the "Shootout" column
merged_results['first_shooter'] = merged_results['first_shooter'].where(merged_results['Shootout'], None)
merged_results['penalty_winner'] = merged_results['penalty_winner'].where(merged_results['Shootout'], None)
merged_results.head(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,game_winner,penalty_winner,first_shooter,Shootout
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,TIE,None,None,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,None,None,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland,None,None,False


In [7]:
# Merge with goalscorers
final_result = pd.merge(merged_results, goalscorers, how='left', on=['date', 'home_team', 'away_team'])
final_result.head(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,game_winner,penalty_winner,first_shooter,Shootout,team,scorer,minute,own_goal,penalty
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,TIE,None,None,False,NaN,NaN,NaN,NaN,NaN
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,None,None,False,NaN,NaN,NaN,NaN,NaN
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland,None,None,False,NaN,NaN,NaN,NaN,NaN


In [8]:
final_result['date'] = pd.to_datetime(final_result['date'])
final_result.head(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,game_winner,penalty_winner,first_shooter,Shootout,team,scorer,minute,own_goal,penalty
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,TIE,None,None,False,NaN,NaN,NaN,NaN,NaN
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England,None,None,False,NaN,NaN,NaN,NaN,NaN
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland,None,None,False,NaN,NaN,NaN,NaN,NaN


In [9]:
# Save the final result to a new CSV file
final_result.to_csv('merged.csv', index=False)